In [1]:
import numpy as np
from numba import njit
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm, Normalize
import pandas as pd
import pickle
from datetime import datetime,timedelta, date
import strax
import straxen
import cutax
from tqdm.notebook import tqdm, trange
strax.utils.tqdm = tqdm
from multihist import Hist1d, Histdd

import copy
from matplotlib import cm
from scipy import stats
import feldman_cousins as fc
from scipy.stats import poisson
from scipy.optimize import minimize, rosen, rosen_der
import h5py

import GOFevaluation as ge
import scipy.stats as sps
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [2]:
import os
style = os.path.join(os.environ['HOME'], 'nton/mplconfigs/stylelib/xenonnt.mplstyle')   
plt.style.use(style)

In [3]:
%matplotlib inline

In [4]:
mpl.rcParams['figure.dpi'] = 130
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 9

In [5]:
%run -i 'YBe_tool.py'
%run -i 'stuff_to_load.py'
mode='3-fold'
# mode='2-fold-only'

In [6]:
Y88_hl=106.6
kbq_initial=698
kbq_sim=389
day_lapse=-np.log(kbq_sim/kbq_initial)/np.log(2)*Y88_hl
#2022-6-1 initial date
source_date=datetime(2022,6,1)+timedelta(days=day_lapse)
dt=(datetime(2022,10,5)-source_date)/timedelta(days=1)

In [6]:
# for i in data_cuts:
#     print(i)

In [8]:
# ddt=75#(datetime(2022,12,5)-datetime(2022,10,5))
# np.exp(-ddt/(Y88_hl/np.log(2)))

In [7]:
hour=183.78

# gamma_ineff=0.9 #NR inefficincy due to gamma coincident
# start_to_mid=0.9809

gamma_ratio=0.93356 #gamma ratio between YBe and Y88 run
# one_week_decay=np.exp(-7/(Y88_hl/np.log(2))) #fitted source decay rate

# sim_ratio=1/(start_to_mid*one_week_decay**2) #old scaling
sim_ratio=1/(np.exp(-dt/(Y88_hl/np.log(2)))) #new scaling

s_hour=15393*sim_ratio #hours of simulation, in the same rate as YBe run
b_hour=142.21*gamma_ratio #hours of background run, in the rate, in the same rate as YBe run
y_hour=365*24
bkg_s=hour/b_hour #pre-scaling for Y88 
sim_s=hour/s_hour #pre-scaling for YBe simulation

In [ ]:
# data_cuts=pd.read_hdf("good_data.hdf5")
bkg=pd.read_hdf("/dali/lgrandi/ghusheng/YBe_data/YBe_superrun/YBe_V1/Y88_event_info_143runs_142.21hrs_all_cuts_with_run_ids_v1.hdf5")
data=pd.read_hdf("/dali/lgrandi/ghusheng/YBe_data/YBe_superrun/YBe_V1/YBe_event_info_239runs_183.78hrs_all_cuts_with_run_ids_v1.hdf5")


# bkg=pd.read_hdf("/dali/lgrandi/ghusheng/YBe_data/YBe_superrun/Y88_event_info_143runs_142.21hrs_all_cuts_with_run_ids.hdf5")
# data=pd.read_hdf("/dali/lgrandi/ghusheng/YBe_data/YBe_superrun/YBe_event_info_239runs_183.78hrs_all_cuts_with_run_ids.hdf5")
# # sim=pd.read_hdf("good_sim.hdf5") #3-fold
# sim_raw=pd.read_csv("/dali/lgrandi/pkavrigin/2022-10-19_YBeNeutron/event_info.csv") #raw sim
sim_raw=pd.read_csv("/dali/lgrandi/xenonnt/simulations/ybe_neutron/ybe_neu_event_info.csv")

In [ ]:
def df_2fold(df):
    return df[(df['cut_s1_tightcoin_3fold']==False)&(df.s1_n_hits==2)]
def df_cut_SA(data_cuts_2only):
    return data_cuts_2only[(data_cuts_2only.cut_position_shadow)&(data_cuts_2only.cut_time_shadow)&(data_cuts_2only.cut_ambience)]

In [10]:
#quality cuts 2fold only
if mode=='2-fold-only':
    ac_h5 = '/dali/lgrandi/xudc/ac_ybe_1109.h5'
    y_hour=365*24
    with h5py.File(ac_h5, 'r', libver='latest', swmr=True) as ipt:
        normalization = ipt['fold2_df'].attrs['normalization']

    ac_sim_2only = pd.read_hdf(ac_h5, key='fold2_df')#, columns=['cs1', 'cs2', 'cut_fiducial_volume', 'cut_near_wires'])
    data_cuts_2only = data.query(' and '.join(targets_2fold[1:]))
    bkg_cuts_2only = bkg.query(' and '.join(targets_2fold[1:]))
    sim_2only = sim_raw.query(' and '.join(targets_2fold[1:]))
    ac_sim_cuts_2only = ac_sim_2only[ac_sim_2only.cut_fiducial_volume]
    s1_bins=np.linspace(1.5,20.5,20)
    data_cuts_2only=df_2fold(data_cuts_2only)
    bkg_cuts_2only=df_2fold(bkg_cuts_2only)
    sim_2only=df_2fold(sim_2only)
    
    # ac_sim_2only=
    # del bkg
    # del data
    # del sim_raw

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [127]:
data_2f_clean=wire_cut(df_cut_SA(data_cuts_2only))
# sim_2f_clean=df_cut_SA(sim_2only)
bkg_2f_clean=wire_cut(df_cut_SA(bkg_cuts_2only))
sim_2f_clean=wire_cut(sim_2only[sim_2only.s1_n_hits==2])

In [59]:
data_2f_clean.s1_n_hits.unique()

array([2], dtype=int16)

In [67]:
#quality cuts 3fold
if mode=='3-fold':
    ac_h5 = '/dali/lgrandi/xudc/ac_ybe_1109.h5'

    with h5py.File(ac_h5, 'r', libver='latest', swmr=True) as ipt:
        normalization = ipt['fold3_df'].attrs['normalization']

    ac_sim = pd.read_hdf(ac_h5, key='fold3_df', columns=['cs1', 'cs2', 'cut_fiducial_volume', 'cut_near_wires','x','y','r','z'])
    data_cuts = data.query(' and '.join(targets_3fold[1:]))
    bkg_cuts = bkg.query(' and '.join(targets_3fold[1:]))
    sim = sim_raw.query(' and '.join(targets_3fold[1:]))
    ac_sim_cuts = ac_sim[ac_sim.cut_fiducial_volume]
    s1_bins=np.linspace(2.5,20.5,19)
    # del bkg
    # del data
    # del sim_raw